# LangChain Part 3: Output Parsing
- Important when using agents with tools e.g. calculator , search engine 
- Many systems expect data in specific way

In [65]:
from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama
from os import getenv
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base="https://openrouter.ai/api/v1",
    model="meta-llama/llama-3.2-3b-instruct:free",
    # temperature=0.2,
)
# llm = ChatOllama(model='llama3.2', temperature=0.2, max_tokens=512)

result = llm.invoke("Hello, how are you today?")
print(result.content)

I'm just a language model, so I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you have. How can I assist you today?


# CSV Parser 
- Used when want to return a list of comma-separated Items

In [49]:
from functools import partial
from pprint import pp
from tempfile import template
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate


output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
print("Format Instructions:\n",format_instructions, "\n")

prompt = PromptTemplate(
    template="List five {subject}. \n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

pprint(prompt.dict())

# Create a model that uses the prompt
chain = prompt | llm | output_parser

# Invoke the model
result = chain.invoke({"subject": "dog breeds"})

print('\nResult:\n',result)




Format Instructions:
 Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz` 

{'_type': 'prompt',
 'input_variables': ['subject'],
 'metadata': None,
 'name': None,
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {'format_instructions': 'Your response should be a list '
                                              'of comma separated values, eg: '
                                              '`foo, bar, baz` or '
                                              '`foo,bar,baz`'},
 'tags': None,
 'template': 'List five {subject}. \n{format_instructions}',
 'template_format': 'f-string',
 'validate_template': False}

Result:
 ['Labrador', 'German Shepherd', 'Poodle', 'Beagle', 'Rottweiler']


# Datetime Parser
- Used when want to return a date in a specific format
- Useful for applications that require a specific date format like a calendar

In [50]:
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate

# Create a new prompt template for datetime parsing
datetime_prompt = PromptTemplate(
    template="What is the current date and time in UTC?\n{format_instructions}",
    input_variables=["format_instructions"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# Create a chain with the new prompt and the existing llm and output_parser
datetime_chain = datetime_prompt | llm | output_parser

# Invoke the chain
datetime_result = datetime_chain.invoke({})

print('\nDatetime Result:\n', datetime_result)



Datetime Result:
 ["I'm a large language model", "I don't have real-time access to the current date and time. However", 'I can suggest ways for you to find the current UTC time.\n\nYou can use an online world clock or a time zone converter website to get the current UTC time. Some popular options include:\n\n- WorldTimeBuddy\n- TimeAndDate\n- UTC Time\n\nPlease note that these websites may not be available in all regions due to internet restrictions.']


# Enum Parser
- Used when want to return a list of items in a specific format
- e.g. list of colors, list of countries
- Useful for applications that require a specific list of items like a dropdown menu.

In [68]:
import re
from langchain.output_parsers.enum import EnumOutputParser
from langchain_core.prompts import PromptTemplate
from enum import Enum
from langchain_core.prompts import PromptTemplate

class EyeColor(Enum):
    RED = "Red"
    GREEN = "Green"
    BLUE = "Blue"
    BROWN = "Brown"
    HAZEL = "Hazel"
    AMBER = "Amber"


parser = EnumOutputParser(enum=EyeColor)




prompt = PromptTemplate.from_template(
    """What color eyes does this person have?

> Person: {person}

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())
chain = prompt | llm | parser

result = chain.invoke({"person": "This person has blue eyes."})

print(result)

EyeColor.BLUE


# Json Parser
- Returns data in JSON format 
- e.g. "{"name": "John", "age": 30, "city": "New York"}"
- Useful for applications that require data in JSON format like APIs, web services

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

# Create a JSON output parser
json_output_parser = JsonOutputParser()

# Define the format instructions
json_format_instructions = json_output_parser.get_format_instructions()
print("Format Instructions:\n", json_format_instructions, "\n")

# Create a prompt template for JSON parsing
json_prompt = PromptTemplate(
    template="Provide the details of a person in JSON format.\n{format_instructions}",
    input_variables=["format_instructions"],
    partial_variables={"format_instructions": json_format_instructions},
)

pprint(json_prompt.dict())


# Create a chain with the new prompt and the existing llm and json_output_parser
json_chain = json_prompt | llm | json_output_parser

# Invoke the chain
json_result = json_chain.invoke({})

print('\nReturn:\n',json_result,'\n')

Format Instructions:
 Return a JSON object. 

{'_type': 'prompt',
 'input_variables': [],
 'metadata': None,
 'name': None,
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {'format_instructions': 'Return a JSON object.'},
 'tags': None,
 'template': 'Provide the details of a person in JSON format.\n'
             '{format_instructions}',
 'template_format': 'f-string',
 'validate_template': False}

Return:
 {'name': 'John Doe', 'age': 30, ' occupation': 'Software Engineer', 'address': {'street': '123 Main St', 'city': 'New York', 'state': 'NY', 'zip': '10001'}, 'interests': ['reading', 'hiking', 'coding'], 'phoneNumbers': [{'type': 'mobile', 'number': '123-456-7890'}, {'type': 'landline', 'number': '555-123-4567'}]} 

content='' additional_kwargs={} response_metadata={} id='run-1b581de9-feed-4b1b-8b53-638f726b2447'
content='Once upon a time, in a small village nestled in the rolling hills of Tuscany, ' additional_kwargs={} response_metadata={} id='run-1b581de9-

In [ ]:
# Streaming 
for s in llm.stream("tell me a story"):
    print(s)
